### Using an external Rust library to speed up lon, lat to [BNG](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) conversion

In [7]:
import numpy as np
import pandas as pd
import math
from ctypes import cdll, c_float, Structure, ARRAY, POINTER, c_int32, c_uint32, c_size_t, c_void_p, cast
from sys import platform
from bng import bng
import pyproj

In [8]:
    if platform == "darwin":
        ext = "dylib"
    else:
        ext = "so"

### Setting up the Rust library. See [here](https://github.com/alexcrichton/rust-ffi-examples/tree/master/python-to-rust) for more

Ensure you've built your Rust library using `cargo build --release`, or the next step will fail.

In [9]:
lib = cdll.LoadLibrary('target/release/liblonlat_bng.' + ext)


class FFITuple(Structure):
    _fields_ = [("a", c_uint32),
                ("b", c_uint32)]

class FFIArray(Structure):
    _fields_ = [("data", c_void_p),
                ("len", c_size_t)]

    # Allow implicit conversions from a sequence of 32-bit unsigned
    # integers.
    @classmethod
    def from_param(cls, seq):
        return seq if isinstance(seq, cls) else cls(seq)

    # Wrap sequence of values. You can specify another type besides a
    # 32-bit unsigned integer.
    def __init__(self, seq, data_type = c_float):
        array_type = data_type * len(seq)
        raw_seq = array_type(*seq)
        self.data = cast(raw_seq, c_void_p)
        self.len = len(seq)

# A conversion function that cleans up the result value to make it
# nicer to consume.
def void_array_to_tuple_list(array, _func, _args):
    return cast(array.data, POINTER(FFITuple * array.len))[0]

convert_vec = lib.convert_vec_c_threaded
convert_vec.argtypes = (FFIArray, FFIArray)
convert_vec.restype = FFIArray
convert_vec.errcheck = void_array_to_tuple_list

def convertbng_list(lons, lats):
    return [(i.a, i.b) for i in iter(convert_vec(lons, lats))]

In [6]:
baz = convert_vec([-0.32824866], [51.44533267])
[(i.a, i.b) for i in iter(baz)]

[(516276L, 173141L)]

### Simple test of average conversion speed, Python version

In [5]:
%%timeit -r 10
bng(
    51.44533267, -0.32824866)

10000 loops, best of 10: 24.1 µs per loop


### Simple test of average conversion speed, Rust 1.0 version

In [8]:
%%timeit -r 10
rust_bng(-0.32824866, 51.44533267)

The slowest run took 1825.23 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 10: 1.42 µs per loop


In [48]:
# UK bounding box
N = 55.811741
E = 1.768960
S = 49.871159
W = -6.379880

### A slightly more realistic test, 100k random points within the UK

In [8]:
df = pd.DataFrame({
        'lon': np.random.uniform(W, E, [100000]),
        'lat': np.random.uniform(N, S, [100000])})

In [10]:
%%timeit -r 10
eastings_northings = df.apply(
    lambda x: bng(x['lat'], x['lon']),
    axis=1)

1 loops, best of 10: 4.11 s per loop


In [9]:
%%timeit -r 10
rust_eastings_northings = df.apply(
    lambda x: rust_bng(c_float(x['lon']), c_float(x['lat'])),
    axis=1)

1 loops, best of 10: 2.05 s per loop


### So far: around a 15x improvement on the simple test, around 2x on the "realistic" test

In [22]:
bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

### Pyproj

In [5]:
%%timeit -r 10
pyproj.transform(wgs84, bng, -0.32824866, 51.44533267)

The slowest run took 7.25 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 10: 8.55 µs per loop


### Using lists instead of a DataFrame

In [49]:
lon_ls = list(np.random.uniform(W, E, [100000]))
lat_ls = list(np.random.uniform(N, S, [100000]))

In [43]:
%%timeit -r 10
res = [rust_bng(i, j) for i, j in zip(lon_ls, lat_ls)]

1 loops, best of 10: 261 ms per loop


In [42]:
%%timeit -r 10
res = [bng(i, j) for i, j in zip(lon_ls, lat_ls)]

1 loops, best of 10: 804 ms per loop


In [23]:
%%timeit -r 10
proj_res = zip(*pyproj.transform(wgs84, bng, lon_ls, lat_ls))

10 loops, best of 10: 93.5 ms per loop


### ~14x speedup over pure Python, ~3x slower than Pyproj

### Let's use the vectorised version

In [52]:
%%timeit -r 10
convertbng_list(lon_ls, lat_ls)

1 loops, best of 10: 194 ms per loop
